## Feature Selection Advanced House Price Prediction

The main aim of this project is to predict the house price based on various features which we will discuss as we go ahead

#### Dataset to downloaded from the below link
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#for feature selection
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
#to visualize the whole data
pd.set_option('display.max_columns', None)


In [2]:
dataset = pd.read_csv('data/X_train_scaled.csv')

In [4]:
dataset.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
0,1,12.247694,0.235294,0.75,0.418208,0.366344,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.036765,0.098361,0.0,0.0,1.0,1.0,0.666667,0.12250,0.666667,1.0,1.00,0.75,0.75,0.25,1.000000,0.125089,0.833333,0.0,0.064212,0.140098,1.0,1.00,1.0,1.0,0.356155,0.413559,0.0,0.577712,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.500000,1.0,0.000000,0.2,0.8,0.046729,0.666667,0.50,0.386460,0.666667,1.0,1.0,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.50,0.666667,0.75,0.0,0.0,0.0
1,2,12.109011,0.000000,0.75,0.495064,0.391317,1.0,1.0,0.000000,0.333333,1.0,0.50,0.0,0.500000,0.2,1.0,0.75,0.6,0.555556,0.875,0.227941,0.524590,0.0,0.0,0.4,0.3,0.333333,0.00000,0.333333,1.0,0.50,0.75,0.75,1.00,0.666667,0.173281,0.833333,0.0,0.121575,0.206547,1.0,1.00,1.0,1.0,0.503056,0.000000,0.0,0.470245,0.000000,0.5,0.666667,0.0,0.375,0.333333,0.333333,0.333333,1.0,0.333333,0.6,0.8,0.289720,0.666667,0.50,0.324401,0.666667,1.0,1.0,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.363636,0.25,0.666667,0.75,0.0,0.0,0.0
2,3,12.317167,0.235294,0.75,0.434909,0.422359,1.0,1.0,0.333333,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.051471,0.114754,0.0,0.0,1.0,1.0,0.666667,0.10125,0.666667,1.0,1.00,0.75,0.75,0.50,1.000000,0.086109,0.833333,0.0,0.185788,0.150573,1.0,1.00,1.0,1.0,0.383441,0.419370,0.0,0.593095,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.333333,1.0,0.333333,0.6,0.8,0.065421,0.666667,0.50,0.428773,0.666667,1.0,1.0,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.727273,0.50,0.666667,0.75,0.0,0.0,0.0
3,4,11.849398,0.294118,0.75,0.388581,0.390295,1.0,1.0,0.333333,0.333333,1.0,0.25,0.0,0.727273,0.4,1.0,0.75,1.0,0.666667,0.500,0.669118,0.606557,0.0,0.0,0.2,0.4,0.333333,0.00000,0.333333,1.0,0.25,0.50,1.00,0.25,0.666667,0.038271,0.833333,0.0,0.231164,0.123732,1.0,0.75,1.0,1.0,0.399941,0.366102,0.0,0.579157,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.666667,0.416667,1.0,0.333333,0.8,0.4,0.074766,0.333333,0.75,0.452750,0.666667,1.0,1.0,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.00,0.666667,0.00,0.0,0.0,0.0
4,5,12.429216,0.235294,0.75,0.513123,0.468761,1.0,1.0,0.333333,0.333333,1.0,0.50,0.0,1.000000,0.4,1.0,0.75,1.0,0.777778,0.500,0.058824,0.147541,0.0,0.0,1.0,1.0,0.666667,0.21875,0.666667,1.0,1.00,0.75,0.75,0.75,1.000000,0.116052,0.833333,0.0,0.209760,0.187398,1.0,1.00,1.0,1.0,0.466237,0.509927,0.0,0.666523,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.666667,0.583333,1.0,0.333333,0.6,0.8,0.074766,0.666667,0.75,0.589563,0.666667,1.0,1.0,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000,0.50,0.666667,0.75,0.0,0.0,0.0


In [5]:
#capture the dependent feature
y_train = dataset[['SalePrice']]

In [6]:
y_train

,SalePrice
0,12.247694
1,12.109011
2,12.317167
3,11.849398
4,12.429216
...,...
1455,12.072541
1456,12.254863
1457,12.493130
1458,11.864462


In [7]:
#drop the dependent data feature from dataset
X_train = dataset.drop(['Id', 'SalePrice'], axis = 1)

# Feature selection

In [8]:
### Apply Feature Selection
# first, I specify the Lasso Regression model, and I
# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.

# Then I use the selectFromModel object from sklearn, which
# will select the features which coefficients are non-zero
feature_sel_model= SelectFromModel(Lasso(alpha=0.005, random_state=0))# remember to set the seed, the random state in this function
feature_sel_model.fit(X_train, y_train)

,estimator,Lasso(alpha=0...andom_state=0)
,threshold,None
,prefit,False
,norm_order,1
,max_features,None
,importance_getter,'auto'
,alpha,0.005
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000


In [13]:
X_train.corr()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
MSSubClass,1.000000,-0.211535,-0.477364,-0.462668,-0.024969,-0.105995,-0.103700,-0.027856,0.022844,-0.053177,-0.025672,-0.024106,-0.027032,-0.042011,-0.340538,0.183527,0.032628,-0.059316,-0.028746,-0.041968,-0.135084,-0.029924,0.005380,-0.018772,0.028615,0.023573,0.016178,-0.030624,0.054655,0.060184,-0.012580,0.051150,0.122078,-0.069836,0.031799,-0.065649,-0.140759,-0.238518,-0.039799,-0.019458,-0.101774,0.023292,-0.288607,0.307886,0.046474,0.081717,0.003491,-0.002333,0.131608,0.177354,-0.023438,0.281721,-0.012383,0.040380,-0.005567,-0.045569,-0.039267,-0.108335,-0.082451,-0.033077,-0.040110,-0.098672,-0.055531,-0.080407,-0.059925,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,0.008610,0.117495,0.028204,-0.007683,-0.013585,-0.021407,-0.021909,-0.029423,-0.015323,-0.016496,0.085357
MSZoning,-0.211535,1.000000,0.315592,0.321035,0.084606,0.221947,0.187166,0.045657,-0.009400,0.095729,0.006208,0.553530,0.122349,0.091497,0.118228,0.187549,0.269302,-0.146350,-0.460736,-0.240519,0.054911,0.035325,0.193364,0.202132,0.147058,0.125409,0.274595,0.151202,0.284754,0.243081,0.087253,0.086154,0.053950,0.170059,-0.046165,0.035922,0.046960,0.237309,0.099890,0.199776,0.254369,0.177141,0.270233,-0.007883,-0.105732,0.192957,0.120119,0.025223,0.266772,0.168978,0.105903,-0.111713,0.247010,0.121840,0.076643,0.170517,0.169354,0.342100,-0.309747,0.285038,0.253433,0.255667,0.176361,0.191037,0.289688,0.125597,0.092446,-0.224789,0.032781,0.024213,0.024663,0.024921,0.040971,0.020753,0.000786,-0.004702,0.010486,0.132537,0.124343,0.092355,0.061642,-0.138661
LotFrontage,-0.477364,0.315592,1.000000,0.653416,-0.034299,0.145917,0.136121,0.040976,-0.002895,0.014149,0.060018,0.235958,-0.002148,-0.024783,0.114807,0.044251,0.204398,-0.031360,-0.086305,-0.058585,0.173115,0.097906,0.050931,0.071570,0.089659,0.140183,0.144067,0.036118,0.074921,0.114524,0.033887,0.132668,0.007358,0.166011,-0.024585,0.038405,0.140659,0.328387,0.010065,0.085454,0.052940,0.043327,0.422404,0.046298,0.024590,0.320432,0.066103,-0.002510,0.162890,0.015991,0.267471,0.013833,0.143600,0.319206,0.028350,0.224109,0.216927,0.266307,-0.038714,0.217442,0.272562,0.317183,0.118507,0.118636,0.078044,0.088199,0.122983,0.018190,0.063526,0.051054,0.107592,0.120527,-0.055172,-0.027281,0.011237,0.019995,0.004364,0.082602,0.069567,0.051355,0.010352,-0.114361
LotArea,-0.462668,0.321035,0.653416,1.000000,-0.105723,0.102652,0.351985,0.135558,-0.023403,0.230379,0.255881,0.245556,0.057468,-0.033856,0.039303,0.009893,0.178215,-0.006305,-0.023095,-0.029424,0.170456,0.201957,0.005881,0.035672,0.030173,0.122350,0.103355,0.011974,0.012252,0.089771,0.031330,0.212134,-0.032171,0.230969,-0.048812,0.092214,0.077247,0.351838,-0.002402,0.047097,0.057671,0.038851,0.467465,0.080465,0.012810,0.385435,0.138273,0.045218,0.179187,0.038717,0.279176,0.001678,0.118496,0.360129,-0.022156,0.327754,0.268215,0.252513,0.020146,0.190834,0.272007,0.322045,0.145790,0.136797,0.023293,0.199846,0.147224,0.010951,0.055700,0.097381,0.091791,0.098180,0.004199,-0.079849,0.047791,0.010521,-0.027164,0.049500,0.055449,0.128188,0.019996,-0.140278
Street,-0.024969,0.084606

In [9]:
feature_sel_model.get_support()

array([ True,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False,  True, False,
       False,  True,  True, False, False, False, False, False, False,
       False, False,  True, False,  True, False, False, False, False,
       False, False, False,  True,  True, False,  True, False, False,
        True,  True, False, False, False, False, False,  True, False,
       False,  True,  True,  True, False,  True,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False])

In [14]:
#let's print the number of total and selected features
#this how we can make a list of the selected features
selected_feat= X_train.columns[(feature_sel_model.get_support())]
#let's print some stats
print(f"total feature: {X_train.shape[1]}")
print(f"selected feature: {len(selected_feat)}")
print(f"features with coefficient shrank to zero: {np.sum(feature_sel_model.estimator_.coef_==0)}")

total feature: 82
selected feature: 21
features with coefficient shrank to zero: 61


In [15]:
selected_feat

Index(['MSSubClass', 'MSZoning', 'Neighborhood', 'OverallQual', 'YearRemodAdd',
       'RoofStyle', 'BsmtQual', 'BsmtExposure', 'HeatingQC', 'CentralAir',
       '1stFlrSF', 'GrLivArea', 'BsmtFullBath', 'KitchenQual', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'PavedDrive',
       'SaleCondition'],
      dtype='object')

In [16]:
X_train = X_train[selected_feat]

In [17]:
X_train.head(5)

,MSSubClass,MSZoning,Neighborhood,OverallQual,YearRemodAdd,RoofStyle,BsmtQual,BsmtExposure,HeatingQC,CentralAir,1stFlrSF,GrLivArea,BsmtFullBath,KitchenQual,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,PavedDrive,SaleCondition
0,0.235294,0.75,0.636364,0.666667,0.098361,0.0,0.75,0.25,1.00,1.0,0.356155,0.577712,0.333333,0.666667,0.000000,0.2,0.8,0.666667,0.50,1.0,0.75
1,0.000000,0.75,0.500000,0.555556,0.524590,0.0,0.75,1.00,1.00,1.0,0.503056,0.470245,0.000000,0.333333,0.333333,0.6,0.8,0.666667,0.50,1.0,0.75
2,0.235294,0.75,0.636364,0.666667,0.114754,0.0,0.75,0.50,1.00,1.0,0.383441,0.593095,0.333333,0.666667,0.333333,0.6,0.8,0.666667,0.50,1.0,0.75
3,0.294118,0.75,0.727273,0.666667,0.606557,0.0,0.50,0.25,0.75,1.0,0.399941,0.579157,0.333333,0.666667,0.333333,0.8,0.4,0.333333,0.75,1.0,0.00
4,0.235294,0.75,1.000000,0.777778,0.147541,0.0,0.75,0.75,1.00,1.0,0.466237,0.666523,0.333333,0.666667,0.333333,0.6,0.8,0.666667,0.75,1.0,0.75
